In [1]:

"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [6]:
df = ssql("""

SELECT t2.*,
       t3.total_game_num,
       t3.duration,
       t2.game_num/t3.total_game_num AS p_
FROM
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'register'
     AND appId IN ('20014',
                   '30014')
     AND date BETWEEN '2020-10-20' AND current_date()- interval 2 DAY
   GROUP BY 1,
            2)t1
JOIN
  (SELECT date,distinct_id,
               playerCount,
               count(distinct_id)AS game_num
   FROM events
   WHERE event = 'gameStart'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-10-20' AND current_date()- interval 2 DAY
   GROUP BY 1,
            2,
            3)t2 ON t1.date = t2.date
AND t1.distinct_id = t2.distinct_id
JOIN
  (SELECT date,distinct_id,
               count(distinct_id)AS total_game_num,
               sum(duration/60000)AS duration
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-10-20' AND current_date()- interval 2 DAY
   GROUP BY 1,
            2)t3 ON t1.date = t3.date
AND t1.distinct_id = t3.distinct_id HAVING t2.game_num/t3.total_game_num >= 0.5
""")

In [8]:
df

,date,duration,playercount,distinct_id,game_num,total_game_num,p_
0,2020-10-26,28.960,10,110183845,3,5,0.600
1,2020-11-05,18.959,6,116495323,3,6,0.500
2,2020-11-02,4.472,10,118007902,1,1,1.000
3,2020-11-08,2.711,10,113919335,1,1,1.000
4,2020-11-13,3.010,10,127899460,1,1,1.000
...,...,...,...,...,...,...,...
2111198,2020-11-06,26.281,10,139485595,4,7,0.571
2111199,2020-11-08,117.962,10,135280887,15,26,0.577
2111200,2020-11-09,23.153,10,119142089,3,5,0.600
2111201,2020-10-28,52.054,10,146594296,6,8,0.750


In [9]:
df_new = df.copy()

In [10]:
df_new.groupby(['date','playercount'])['total_game_num','duration'].describe()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


total_game_num                                      \
                                count      mean        std  min  25%  50%   
date       playercount                                                      
2020-10-20 4                     48.0  4.250000   7.153336  1.0  1.0  2.0   
           5                     36.0  6.500000   9.222643  1.0  1.0  2.0   
           6                     39.0  2.282051   1.605070  1.0  1.0  2.0   
           7                     43.0  2.093023   1.411078  1.0  1.0  2.0   
           8                     77.0  3.389610   7.411151  1.0  1.0  2.0   
...                               ...       ...        ...  ...  ...  ...   
2020-12-02 6                   1288.0  7.254658  11.017626  1.0  2.0  2.0   
           7                   1458.0  6.056927   8.569642  1.0  2.0  3.0   
           8                   1784.0  4.427691   5.298529  1.0  2.0  2.0   
           9                   2044.0  4.040607   5.243910  1.0  1.0  2.0   
           10                  2704.0  7.440828  13.748636  1.0  1.0  2.0   

                                   duration                               \
                        75%    max    count       mean        std    min   
date       playercount                                                     
2020-10-20 4            4.0   32.0     48.0  11.825708  16.358205  0.535   
           5            6.0   33.0     36.0  20.105472  24.807133  0.713   
           6            2.0    9.0     39.0  11.393077   8.144296  0.976   
           7            2.0    8.0     43.0  10.429372   6.099757  0.634   
           8            2.0   56.0     77.0  18.660013  40.821192  0.781   
...                     ...    ...      ...        ...        ...    ...   
2020-12-02 6            8.0   91.0   1288.0  21.608873  30.862550  0.007   
           7            6.0   76.0   1458.0  20.992260  27.284547  0.655   
           8            5.0   46.0   1784.0  20.029658  23.944608  0.626   
           9            4.0   62.0   2044.0  20.920992  27.161880  0.635   
           10           6.0  122.0   2704.0  43.209992  81.514897  0.007   

                                                             
                            25%      50%       75%      max  
date       playercount                                       
2020-10-20 4            3.41100   6.6050  11.98900   78.802  
           5            5.28700   9.0310  22.95150  107.052  
           6            5.79750   8.8690  14.65900   41.014  
           7            6.09200   9.3290  12.42950   31.607  
           8            4.95400  10.1930  17.12600  325.034  
...                         ...      ...       ...      ...  
2020-12-02 6            4.86100   9.4335  24.38025  317.612  
           7            5.51025  10.5560  22.70050  192.300  
           8            6.15925  11.2805  23.09875  197.884  
           9            6.61875  11.3625  22.88700  386.981  
           10           8.26975  14.6560  37.29050  708.153  

[308 rows x 16 columns]

In [11]:
df_new.groupby(['date','playercount'])['total_game_num','duration'].describe().to_clipboard()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [15]:
df_old = ssql("""
SELECT t2.*,
       t3.total_game_num,
       t3.duration,
       t2.game_num/t3.total_game_num AS p_
FROM
  (SELECT t5.date,
          t5.distinct_id
   FROM
     (SELECT date,distinct_id
      FROM events
      WHERE event = 'register'
        AND appId IN ('20014',
                      '30014')
        AND date BETWEEN '2020-10-20' AND current_date()- interval 2 DAY
      GROUP BY 1,
               2)t4
   RIGHT JOIN
     (SELECT date,distinct_id
      FROM events
      WHERE event = 'login'
        AND appId IN ('20014',
                      '30014')
        AND date BETWEEN '2020-10-20' AND current_date()- interval 2 DAY
      GROUP BY 1,
               2)t5 ON t4.date = t5.date
   AND t4.distinct_id = t5.distinct_id
   WHERE t4.distinct_id IS NULL)t1
JOIN
  (SELECT date,distinct_id,
               playerCount,
               count(distinct_id)AS game_num
   FROM events
   WHERE event = 'gameStart' 
     AND gameTypeId = 1800
     AND date BETWEEN '2020-10-20' AND current_date()- interval 2 DAY
   GROUP BY 1,
            2,
            3)t2 ON t1.date = t2.date
AND t1.distinct_id = t2.distinct_id
JOIN
  (SELECT date,distinct_id,
               count(distinct_id)AS total_game_num,
               sum(duration/60000)as duration
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-10-20' AND current_date()- interval 2 DAY
   GROUP BY 1,
            2)t3 ON t1.date = t3.date
AND t1.distinct_id = t3.distinct_id HAVING t2.game_num/t3.total_game_num >= 0.5
""")

In [16]:
df_old

,date,duration,playercount,distinct_id,game_num,total_game_num,p_
0,2020-11-22,9.873,8,118623018,1,1,1.000
1,2020-11-25,16.595,7,140773186,3,6,0.500
2,2020-11-06,15.849,10,129576826,2,4,0.500
3,2020-11-15,8.969,9,144974515,1,2,0.500
4,2020-11-15,8.969,10,144974515,1,2,0.500
...,...,...,...,...,...,...,...
5365924,2020-11-14,14.273,6,117888642,2,3,0.667
5365925,2020-11-28,4.371,10,128660993,1,1,1.000
5365926,2020-12-02,43.031,10,113618870,5,6,0.833
5365927,2020-11-06,25.718,10,106676964,2,3,0.667


In [17]:
df_old.groupby(['date','playercount'])['total_game_num','duration'].describe()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


total_game_num                                       \
                                count       mean        std  min  25%  50%   
date       playercount                                                       
2020-10-20 4                     38.0   3.315789   5.946179  1.0  1.0  2.0   
           5                     39.0   2.410256   1.665722  1.0  1.5  2.0   
           6                     41.0   2.268293   1.533369  1.0  1.0  2.0   
           7                     44.0   2.772727   3.820773  1.0  1.0  2.0   
           8                     99.0   4.111111   5.275838  1.0  1.5  2.0   
...                               ...        ...        ...  ...  ...  ...   
2020-12-02 6                   5842.0   5.231770   7.108764  1.0  2.0  2.0   
           7                   7255.0   4.804273   6.550288  1.0  2.0  2.0   
           8                  10109.0   4.408052   5.677330  1.0  2.0  2.0   
           9                  13403.0   4.464150   5.369468  1.0  2.0  2.0   
           10                 36960.0  11.581791  14.862918  1.0  2.0  6.0   

                                    duration                               \
                         75%    max    count       mean        std    min   
date       playercount                                                      
2020-10-20 4             3.0   37.0     38.0  11.188500  17.851470  1.366   
           5             2.5    9.0     39.0   9.962256   7.217520  1.609   
           6             3.0    8.0     41.0  15.741220  26.798437  1.163   
           7             2.0   21.0     44.0  13.922841  16.201038  2.791   
           8             4.0   32.0     99.0  25.000061  31.664258  1.093   
...                      ...    ...      ...        ...        ...    ...   
2020-12-02 6             6.0   80.0   5842.0  16.740751  22.031018  0.007   
           7             5.0   94.0   7255.0  17.663208  23.024169  0.396   
           8             5.0  104.0  10109.0  20.187170  26.267434  0.016   
           9             5.0  100.0  13403.0  24.278855  30.979141  0.558   
           10           14.0  213.0  36960.0  69.917299  90.815254  0.650   

                                                               
                             25%      50%       75%       max  
date       playercount                                         
2020-10-20 4             3.34000   6.1300  12.79600   107.083  
           5             4.82450   8.7120  12.72950    35.707  
           6             4.85200   9.0860  14.09300   128.303  
           7             5.41825  10.1395  14.41025    73.456  
           8             8.21050  13.6870  28.00350   185.575  
...                          ...      ...       ...       ...  
2020-12-02 6             5.07300   9.4510  18.88850   305.799  
           7             5.62250  10.2010  19.97800   299.154  
           8             6.64700  11.9130  23.38400   437.629  
           9             7.83600  14.3000  28.51350   517.961  
           10           14.38475  36.8150  87.68600  1250.843  

[310 rows x 16 columns]

In [18]:
df_old.groupby(['date','playercount'])['total_game_num','duration'].describe().to_clipboard()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [2]:
df = ssql("""SELECT t2.date,
       t2.distinct_id,
       t1.game_level,
       t1.date,
       datediff(t1.date, t2.date)AS date_gap
FROM
  (SELECT date,distinct_id,
               game_level,
               time
   FROM events
   WHERE event = 'upgrade'
     AND date BETWEEN '2020-11-30' AND current_date()
     AND gameTypeId = 1800
   GROUP BY 1,
            2,
            3,
            4) t1
RIGHT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'register'
     AND date BETWEEN '2020-11-30' AND current_date()
     AND appId IN ('20014',
                   '30015')
   GROUP BY 1,
            2)t2 ON t1.distinct_id = t2.distinct_id
ORDER BY 1,
         2,
         3""")

In [3]:
df

,date,distinct_id,date_gap,game_level
0,2020-11-30,100021349,NaN,NaN
1,2020-11-30,100054823,NaN,NaN
2,2020-12-06,100119851,6.0,1.0
3,2020-11-30,10019865,NaN,NaN
4,2020-11-30,100619346,0.0,1.0
...,...,...,...,...
1637858,2020-12-09,95598002,NaN,NaN
1637859,2020-12-09,98287149,NaN,NaN
1637860,2020-12-09,98377805,0.0,1.0
1637861,2020-12-09,98377805,0.0,2.0


In [4]:
df.groupby(['game_level'])['date_gap'].describe()

,count,mean,std,min,25%,50%,75%,max
game_level,,,,,,,,
1.0,333503.0,0.055973,0.364537,0.0,0.0,0.0,0.0,9.0
2.0,292261.0,0.111753,0.505621,0.0,0.0,0.0,0.0,9.0
3.0,222175.0,0.261344,0.743171,0.0,0.0,0.0,0.0,9.0
4.0,175849.0,0.444120,0.924745,0.0,0.0,0.0,1.0,9.0
5.0,135058.0,0.679930,1.092323,0.0,0.0,0.0,1.0,9.0
6.0,101742.0,0.942541,1.225156,0.0,0.0,1.0,1.0,9.0
7.0,75709.0,1.197902,1.335391,0.0,0.0,1.0,2.0,9.0
8.0,55844.0,1.457399,1.439089,0.0,1.0,1.0,2.0,9.0
9.0,40182.0,1.709746,1.527852,0.0,1.0,1.0,2.0,9.0


In [5]:
df.groupby(['game_level'])['date_gap'].describe().to_clipboard()